In [17]:
n = 3
large_encoder = True

In [18]:
import os
import json 
import glob
from pathlib import Path

In [19]:
def get_sents(file):
    with open(file, 'r') as f:
        for line in f:
            doc = json.loads(line)
            sents = list()
            if isinstance(doc['knowledge_graph']['title'][0]['value'], str):
                sents.append(doc['knowledge_graph']['title'][0]['value'])
                sents.extend(doc['split_sentences'])
                for sent in sents:
                    sent = str(sent).replace('\n', ' ').replace('\t', ' ')
                    while '  ' in sent:
                        sent.replace('  ', ' ')
                    yield sent

In [20]:
project_dir = Path(f'TestProject00{n}/')
os.makedirs(project_dir, exist_ok=True)

jl_files = ['Data/sample_news_2019-03-04.jl', 'Data/sample_news_2019-03-05.jl', 'Data/sample_news_2019-03-06.jl']
jl_files

['Data/sample_news_2019-03-04.jl',
 'Data/sample_news_2019-03-05.jl',
 'Data/sample_news_2019-03-06.jl']

In [ ]:
for jl in jl_files:
    fname = Path(jl).stem
    dump_tsv = project_dir/f'{fname}.tsv'
    with open(dump_tsv, 'x') as dt:
        sent_gen = get_sents(jl)
        for i, sent in enumerate(sent_gen):
            dt.write(f'{i}\t{sent}\n')

In [ ]:
tsv_files = glob.glob(f'{project_dir}/*.tsv')
tsv_files

In [9]:
from SimSent.indexer.index_builder import IndexBuilder
from SimSent.vectorizer.sentence_vectorizer import SentenceVectorizer

W0425 12:12:52.222565 4399863232 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [ ]:
sv = SentenceVectorizer(large=large_encoder)

In [ ]:
ibdr = IndexBuilder(project_dir, sentence_vectorizer=sv)

In [ ]:
for tsv in tsv_files:
    ibdr.tsv_to_index(tsv)

In [ ]:
from SimSent.server.service_handler import QueryHandler

from SimSent.vectorizer.sentence_vectorizer import DockerVectorizer
from SimSent.indexer.deploy_handler import RangeShards

In [ ]:
# Ensure correct docker is running the vectorizer container in background 
dv = DockerVectorizer(large=large_encoder)
rs = RangeShards(project_dir, nprobe=8, get_nested=True)

In [ ]:
qp = QueryHandler(dv, rs, project_dir=project_dir, nested=True)

In [ ]:
keys = list(rs.shards.keys())
keys

In [ ]:
query = ''

In [ ]:
qp.query_corpus(query, keys)